# Dataset Exploration

In [1]:
from torch_geometric.datasets import KarateClub

dataset = KarateClub()
print(f'Dataset: {dataset}')
print(f'===================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')
print(f'Number of classes: {dataset[0]}')

Dataset: KarateClub()
Number of graphs: 1
Number of features: 34
Number of classes: 4
Number of classes: Data(x=[34, 34], edge_index=[2, 156], y=[34], train_mask=[34])


In [2]:
from torch_sparse import SparseTensor
row, col = dataset[0].edge_index
N = dataset[0].num_nodes

adj = SparseTensor(row=row, col=col, sparse_sizes=(N, N))
print(row.shape, col.shape)

# dataset[0].edge_index
# adj
# set_diag(adj)

torch.Size([156]) torch.Size([156])


In [3]:
type(dataset[0].edge_index)

torch.Tensor

In [4]:
data = dataset[0]
data

Data(x=[34, 34], edge_index=[2, 156], y=[34], train_mask=[34])

# GAT Model Creation

In [5]:
from Classes.GAT import GATConv
import os
path = os.getcwd()
print(path)

/home/vimukthi/MyProjects/Machine-Learning/Graph_ML
/home/vimukthi/MyProjects/Machine-Learning/Graph_ML


In [6]:
import torch
import torch.nn.functional as F
import torch.nn as nn

class GAT(nn.Module):
    def __init__(self, in_features, out_features, heads=1):
        super().__init__()
        self.conv1 = GATConv(in_features, out_features, heads)
        self.conv2 = GATConv(heads * out_features, dataset.num_classes, heads)
        
    def forward(self, x, edge_index):
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        return x

In [7]:
model = GAT(in_features=34, out_features=16)
print(model)

GAT(
  (conv1): GATConv(34, 16, heads=1)
  (conv2): GATConv(16, 4, heads=1)
)


In [8]:
data

Data(x=[34, 34], edge_index=[2, 156], y=[34], train_mask=[34])

In [9]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()
lossArr = []
num_epochs = 10

def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss

for epoch in range(num_epochs):
    loss = train()
    lossArr.append(loss.item())
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

Epoch: 000, Loss: 1.3843
Epoch: 001, Loss: 1.4545
Epoch: 002, Loss: 1.4109
Epoch: 003, Loss: 1.4288
Epoch: 004, Loss: 1.4191
Epoch: 005, Loss: 1.4068
Epoch: 006, Loss: 1.3683
Epoch: 007, Loss: 1.3694
Epoch: 008, Loss: 1.4074
Epoch: 009, Loss: 1.3300


## Original GAT

In [10]:
from torch_geometric.nn import GATConv

class GAT(nn.Module):
    def __init__(self, hidden_channels, heads):
        super().__init__()
        torch.manual_seed(1234)
        self.conv1 = GATConv(data.num_features, hidden_channels, heads)
        self.conv2 = GATConv(heads * hidden_channels, dataset.num_classes, heads)
        
    def forward(self, x, edge_index):
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        return x
    
model = GAT(hidden_channels=16, heads=8)    
print(model)

GAT(
  (conv1): GATConv(34, 16, heads=8)
  (conv2): GATConv(128, 4, heads=8)
)


In [11]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()
lossArr = []
num_epochs = 10

def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss

for epoch in range(num_epochs):
    loss = train()
    lossArr.append(loss.item())
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

Epoch: 000, Loss: 3.4951
Epoch: 001, Loss: 3.3172
Epoch: 002, Loss: 3.2252
Epoch: 003, Loss: 3.0916
Epoch: 004, Loss: 3.0681
Epoch: 005, Loss: 2.8321
Epoch: 006, Loss: 2.6403
Epoch: 007, Loss: 2.6831
Epoch: 008, Loss: 2.2614
Epoch: 009, Loss: 2.2220
